# Navigator OAI Endpoints with the Gretel SDK

In this blueprint, we'll be able to make requests to a OpenAI compliant Navigator hosted on AWS or Azure

In [ ]:
%%capture
!pip install gretel-client

In [ ]:
import pandas as pd

from gretel_client import Gretel

PROMPT = """Generate a mock dataset for users from the Foo company based in France.
Each user should have the following columns:
* first_name: traditional French first names.
* last_name: traditional French surnames. 
* email: formatted as the first letter of their first name followed by their last name @foo.io (e.g., jdupont@foo.io)
* gender: Male/Female
* city: a city in France
* country: always 'France'.
"""

EDIT_PROMPT = """Edit the table and add the following columns:
* occupation: a random occupation
* education level: make it relevant to the occupation
"""

table_headers = ["first_name", "last_name", "email", "gender", "city", "country"]
table_data = [
    {
        "first_name": "Lea",
        "last_name": "Martin",
        "email": "lmartin@foo.io",
        "gender": "Female",
        "city": "Lyon",
        "country": "France",
    }
]

SAMPLE_DATA = pd.DataFrame(table_data, columns=table_headers)

STREAM = True

## Azure

To get started, you'll need to update your OpenAI endpoint and API key as appropriate

In [ ]:
from openai import OpenAI

endpoint = "my-domain.com"
api_key = "abc"
# This is used for local testing since the AzureOpenAI class requires an actual deployed Azure OpenAI endpoint
client = OpenAI(
    base_url=f"https://{endpoint}/v1/inference/oai/v1",
    api_key=api_key
)

MODEL = "gretelai-azure/gpt-3.5-turbo"

azure_open_ai = Gretel.create_navigator_azure_oai_adapter(client)

usage, generated_df = azure_open_ai.generate(
    MODEL,
    PROMPT,
    num_records=10,
    sample_data=SAMPLE_DATA,
    stream=STREAM,
)

print(generated_df)
print("*****")
print(usage)

# Let's edit this now

EDIT_PROMPT = """Edit the table and add the following columns:
* Occupation: a random occupation
* Education level: make it relevant to the occupation
"""

usage, edited_df = azure_open_ai.edit(
    MODEL,
    EDIT_PROMPT,
    seed_data=generated_df,
    stream=STREAM
)
print(edited_df)
print("*****")
print(usage)


In [ ]:
# Let's do the same thing again, but now we're gonna use streaming mode. In order to have streaming mode 
# utilize a Python generator, we have to disable DataFrame mode.

# In this mode, we only get back a single Python generator. Because of how the OAI API spec is designed, 
# usage information will only come back after the entire stream has been exhausted.
data_stream = azure_open_ai.generate(
    MODEL,
    PROMPT,
    num_records=20,
    sample_data=SAMPLE_DATA,
    as_dataframe=False,
)

generated_records = []

for record in data_stream:
    print(record)
    generated_records.append(record)
    
# Once the stream is exhausted, we capture our usage/metadata and make it
# available as an attribute on our data stream
print("*****")
print(data_stream.metadata)

# Let's edit this now

data_stream = azure_open_ai.edit(
    MODEL,
    EDIT_PROMPT,
    seed_data=generated_records,
    as_dataframe=False
)

for record in data_stream:
    print(record)

print("*****")
print(data_stream.metadata)

## AWS

To get started here, you'll need to update your 
* profile name
* sagemaker endpoint name
* aws region

In [ ]:
import boto3

profile_name = "my-profile-name"
sagemaker_endpoint = "my-sagemaker-endpoint"
aws_region = "us-east-1"

boto3.setup_default_session(profile_name=profile_name)

sagemaker_client = boto3.client("sagemaker-runtime", region_name=aws_region)

sagemaker = Gretel.create_navigator_sagemaker_adapter(sagemaker_client, sagemaker_endpoint)

MODEL = "gretelai/auto"

usage, generated_df = sagemaker.generate(
    MODEL,
    PROMPT,
    num_records=10,
    sample_data=SAMPLE_DATA,
    stream=STREAM,
)

print(generated_df)
print("*****")
print(usage)

# Edit!

usage, edited_df = sagemaker.edit(
    MODEL,
    EDIT_PROMPT,
    seed_data=generated_df,
    stream=STREAM
)
print(edited_df)
print("*****")
print(usage)